<a href="https://colab.research.google.com/github/neeharika0603/Lab5/blob/main/crypto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from PIL import Image

def psnr(original, contrast):
    mse = np.mean((original - contrast) ** 2)
    if mse == 0:
        return 100
    PIXEL_MAX = 255.0
    PSNR = 20 * np.log10(PIXEL_MAX / np.sqrt(mse))
    return PSNR

def normxcorr2D(image, template):
    t = np.asarray(template, dtype=np.float64)
    t = t - np.mean(t)
    norm = np.sqrt(np.sum(np.square(t)))
    if norm == 0:
        raise ValueError("Norm of the input is 0")
    t = t / norm

    sum_filter = np.ones(np.shape(t))
    a = np.asarray(image, dtype=np.float64)
    aa = np.square(a)

    a_sum = np.sum(a)
    aa_sum = np.sum(aa)

    numer = np.sum(a * t)
    denom = np.sqrt(aa_sum - np.square(a_sum) / np.size(t))

    tol = np.sqrt(np.finfo(denom.dtype).eps)
    nxcorr = np.where(denom < tol, 0, numer / denom)

    nxcorr = np.where(np.abs(nxcorr - 1.) > np.sqrt(np.finfo(nxcorr.dtype).eps), nxcorr, 0)

    return np.mean(nxcorr)

def encrypt(input_image, share_size):
    image = np.asarray(input_image)
    (row, column, depth) = image.shape
    shares = np.random.randint(0, 256, size=(row, column, depth, share_size))
    shares[:, :, :, -1] = image.copy()
    for i in range(share_size - 1):
        shares[:, :, :, -1] = shares[:, :, :, -1] ^ shares[:, :, :, i]

    return shares, image

def decrypt(shares):
    (row, column, depth, share_size) = shares.shape
    shares_image = shares.copy()
    for i in range(share_size - 1):
        shares_image[:, :, :, -1] = shares_image[:, :, :, -1] ^ shares_image[:, :, :, i]

    final_output = shares_image[:, :, :, share_size - 1]
    output_image = Image.fromarray(final_output.astype(np.uint8))
    return output_image, final_output

if __name__ == "__main__":
    print("Save input image as 'Input.jpg' in the same folder as this file\n")

    try:
        share_size = int(input("Input the number of shares images you want to create for encrypting (min is 2, max is 8) : "))
        if share_size < 2 or share_size > 8:
            raise ValueError
    except ValueError:
        print("Input is not a valid integer!")
        exit(0)

    try:
        input_image = Image.open('Input.jpg')  # Corrected file extension to 'Input.jpg'
    except FileNotFoundError:
        print("Input file not found!")
        exit(0)

    print("Image uploaded successfully!")
    print("Input image size (in pixels) : ", input_image.size)
    print("Number of shares image = ", share_size)

    shares, input_matrix = encrypt(input_image, share_size)

    for ind in range(share_size):
        image = Image.fromarray(shares[:, :, :, ind].astype(np.uint8))
        name = "XOR_Share_" + str(ind + 1) + ".png"
        image.save(name)

    output_image, output_matrix = decrypt(shares)

    output_image.save('XOR.png')
    print("Image is saved 'XOR.png' ...")

    print("Evaluation metrics : ")
    print(f"PSNR value is {psnr(input_matrix, output_matrix)} dB")
    print(f"Mean NCORR value is {normxcorr2D(input_matrix, output_matrix)}")


Save input image as 'Input.jpg' in the same folder as this file

